# <center>__Data Cleaning Assignment__</center>

In [1]:
"""import necessary packages"""

import re
import numpy as np
import pandas as pd
import missingno as msngo
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


## *Data Cleaning*

### Objectives
- Get column names from metadata file (census_income_metadata.txt) and apply to census data
- Inspect the data
    - Inspect text data and clean/normalize where applicable
    - Determine unique values (levels) for categorical data
- Handle missing data appropriately
- Ensure proper datatype for each feature

In [2]:
"""read in the data"""

census_income_train = pd.read_csv('census_income_train.csv', header=None)
census_income_test = pd.read_csv('census_income_test.csv', header=None)

In [3]:
"""create a dictionary to store feature names and associated metadata"""

metadata = open('census_income_metadata.txt')
metadata = metadata.readlines()[142:184]
metadata_dict = {}
for line in metadata:
    key = re.search(r'.*(?=:)', line).group().replace(" ", '_')
    key = re.sub(f'[^\w\d_]', '', key)
    value = re.search(r'(?<=:).*', line).group()
    value = re.sub(r'[\n\.]', '', value)
    value = value.strip()
    value = [val.strip() for val in value.split(',')]
    metadata_dict[key] = value
metadata_dict['outcome'] = ['- 50000', '50000+']

In [4]:
"""
get column names from metadata dict and apply to train and test data
drop instance weight feature form training and test sets
colnames list will be re-used durig data cleaning process, remove instance_weight from list
"""

colnames = list(metadata_dict.keys())
colnames.remove('_instance_weight')
census_income_train.columns = colnames
census_income_test.columns = colnames
census_income_train.drop(columns=['instance_weight'], inplace=True)
census_income_test.drop(columns=['instance_weight'], inplace=True)
colnames.remove('instance_weight')

Clean Text Data

In [5]:
"""
check dtype to identify columns containing text data
remove . in target, strip whitespace
"""
for col in colnames:
    if census_income_train[col].dtype == 'object':
        census_income_train[col] = census_income_train[col].str.replace('.', '', regex=False)
        census_income_train[col] = census_income_train[col].str.strip()
    if census_income_test[col].dtype == 'object':
        census_income_test[col] = census_income_test[col].str.replace('.', '', regex=False)
        census_income_test[col] = census_income_test[col].str.strip()

In [6]:
"""
create list of proper data types for each feature as specified in census_income_metadata.txt
"""

continuous_features = ['age', 'wage_per_hour', 'capital_gains', 'capital_losses',
                       'dividends_from_stocks', 'num_persons_worked_for_employer',
                       'weeks_worked_in_year']
nominal_features = [feature for feature in colnames if feature not in continuous_features]

Check Unique Values in Data

In [7]:
"""
instantiate dictionaries to store problem values idnetified in train and test sets
problem values will be determined by comparing unique values in the data to known set of unique values specified
in census_income_metadata.txt
"""
# create dictionaries
train_problem_values_dict = {}
test_problem_values_dict = {}
# get unique values from each column in train and test sets, compute set difference between actual and expected unique values
for col in nominal_features:
    unique_vals_train = set([str(val) for val in census_income_train[col].unique()])
    unique_vals_test = set([str(val) for val in census_income_test[col].unique()])
    unique_vals_metadata = set(metadata_dict[col])
    train_problem_values_dict[col] = list(unique_vals_train - unique_vals_metadata)
    test_problem_values_dict[col] = list(unique_vals_test - unique_vals_metadata)

In [8]:
"""print problem values idntified in training set"""
print(train_problem_values_dict)

{'class_of_worker': [], 'detailed_industry_recode': [], 'detailed_occupation_recode': [], 'education': [], 'enroll_in_edu_inst_last_wk': [], 'marital_stat': [], 'major_industry_code': [], 'major_occupation_code': [], 'race': [], 'hispanic_origin': [], 'sex': [], 'member_of_a_labor_union': [], 'reason_for_unemployment': [], 'full_or_part_time_employment_stat': [], 'tax_filer_stat': [], 'region_of_previous_residence': [], 'state_of_previous_residence': ['?'], 'detailed_household_and_family_stat': [], 'detailed_household_summary_in_household': [], 'migration_codechange_in_msa': ['?'], 'migration_codechange_in_reg': ['?'], 'migration_codemove_within_reg': ['?'], 'live_in_this_house_1_year_ago': [], 'migration_prev_res_in_sunbelt': ['?'], 'family_members_under_18': [], 'country_of_birth_father': ['?'], 'country_of_birth_mother': ['?'], 'country_of_birth_self': ['?'], 'citizenship': [], 'own_business_or_self_employed': [], 'fill_inc_questionnaire_for_veterans_admin': [], 'veterans_benefits':

In [9]:
"""print problem values identified in the test set"""
print(test_problem_values_dict)

{'class_of_worker': [], 'detailed_industry_recode': [], 'detailed_occupation_recode': [], 'education': [], 'enroll_in_edu_inst_last_wk': [], 'marital_stat': [], 'major_industry_code': [], 'major_occupation_code': [], 'race': [], 'hispanic_origin': [], 'sex': [], 'member_of_a_labor_union': [], 'reason_for_unemployment': [], 'full_or_part_time_employment_stat': [], 'tax_filer_stat': [], 'region_of_previous_residence': [], 'state_of_previous_residence': ['?'], 'detailed_household_and_family_stat': [], 'detailed_household_summary_in_household': [], 'migration_codechange_in_msa': ['?'], 'migration_codechange_in_reg': ['?'], 'migration_codemove_within_reg': ['?'], 'live_in_this_house_1_year_ago': [], 'migration_prev_res_in_sunbelt': ['?'], 'family_members_under_18': [], 'country_of_birth_father': ['?'], 'country_of_birth_mother': ['?'], 'country_of_birth_self': ['?'], 'citizenship': [], 'own_business_or_self_employed': [], 'fill_inc_questionnaire_for_veterans_admin': [], 'veterans_benefits':

Comparison of unique values in the training and test sets to unique values in census_income_metadata.txt shows the existence of a single problem value "?." This value will be treated as missing and replaced with "None."

In [10]:
"""change records with ? to None dtype"""
census_income_train = census_income_train.applymap(lambda x : None if x == '?' else x)
census_income_test = census_income_test.applymap(lambda x : None if x == '?' else x)

In [11]:
"""verify ? has been replaced with None in train set"""

'?' in census_income_train.values

False

In [12]:
"""verify ? has been replaced with None in test set"""

'?' in census_income_test.values

False

Check Missing Values and Missingness Mechanism in Data

In [13]:
"""print count of missing continuous features in train set"""

census_income_train[continuous_features].isnull().sum()

age                                0
wage_per_hour                      0
capital_gains                      0
capital_losses                     0
dividends_from_stocks              0
num_persons_worked_for_employer    0
weeks_worked_in_year               0
dtype: int64

In [14]:
"""print count of missing continuous features in test set"""

census_income_train[continuous_features].isnull().sum()

age                                0
wage_per_hour                      0
capital_gains                      0
capital_losses                     0
dividends_from_stocks              0
num_persons_worked_for_employer    0
weeks_worked_in_year               0
dtype: int64

In [15]:
"""print count of missing nominal features in train set"""

census_income_train[nominal_features].isnull().sum()

class_of_worker                                  0
detailed_industry_recode                         0
detailed_occupation_recode                       0
education                                        0
enroll_in_edu_inst_last_wk                       0
marital_stat                                     0
major_industry_code                              0
major_occupation_code                            0
race                                             0
hispanic_origin                                  0
sex                                              0
member_of_a_labor_union                          0
reason_for_unemployment                          0
full_or_part_time_employment_stat                0
tax_filer_stat                                   0
region_of_previous_residence                     0
state_of_previous_residence                    708
detailed_household_and_family_stat               0
detailed_household_summary_in_household          0
migration_codechange_in_msa    

In [16]:
"""print count of missing nominal features in test set"""

census_income_test[nominal_features].isnull().sum()

class_of_worker                                  0
detailed_industry_recode                         0
detailed_occupation_recode                       0
education                                        0
enroll_in_edu_inst_last_wk                       0
marital_stat                                     0
major_industry_code                              0
major_occupation_code                            0
race                                             0
hispanic_origin                                  0
sex                                              0
member_of_a_labor_union                          0
reason_for_unemployment                          0
full_or_part_time_employment_stat                0
tax_filer_stat                                   0
region_of_previous_residence                     0
state_of_previous_residence                    330
detailed_household_and_family_stat               0
detailed_household_summary_in_household          0
migration_codechange_in_msa    

In [17]:
"""drop columns missing more than 30% of data from train set"""

perc = 0.70
threshold = int(perc*census_income_train.shape[0])
census_income_train.dropna(axis=1, thresh=threshold, inplace=True)
# update nominal features index to account for dropped features
nominal_features = [feature for feature in census_income_train.columns.values.tolist() if feature not in continuous_features]

In [18]:
"""drop columns missing more than 30% of data from test set"""

perc = 0.70
threshold = int(perc*census_income_test.shape[0])
census_income_test.dropna(axis=1, thresh=threshold, inplace=True)
# update nominal features index to account for dropped features
nominal_features = [feature for feature in census_income_test.columns.values.tolist() if feature not in continuous_features]

In [ ]:
"""determine total percentage of records with at least one missing attribute train set"""

census_income_train.isnull().any(axis=1).mean()*100

In [ ]:
"""determine total percentage of records with at least one missing attribute test set"""

census_income_test.isnull().any(axis=1).mean()*100

In [ ]:
"""
print missing matrix for train set using missingo module
missing matrix will provide high-level view of missingness mechanism (MCAR, MAR, NMAR)
"""
msngo.matrix(census_income_train)

In [ ]:
"""
print missing matrix for test set using missingo module
missing matrix will provide high-level view of missingness mechanism (MCAR, MAR, NMAR)
"""

msngo.matrix(census_income_test)

Examining the missingness matrix it's clear that missing data in both the training and test sets is not missing completely at random (MCAR). Since less the 5% of records are missing data and the misssingness mechanism is not MCAR remaining records with misssing attributes will be dropped.

In [ ]:
"""drop records with missing attributes from train set"""
census_income_train.dropna(axis=0, inplace=True)
census_income_train.isnull().sum()

In [ ]:
"""drop records with missing attributes from test set"""
census_income_test.dropna(axis=0, inplace=True)
census_income_test.isnull().sum()

In [ ]:
"""check data types census_income_test"""
census_income_train[continuous_features].dtypes

In [ ]:
"""check data types census_income_test"""

census_income_test[continuous_features].dtypes

In [ ]:
"""convert nominal features to type category, census_income_train"""
census_income_train[nominal_features] = census_income_train[nominal_features].astype('category')
census_income_train[nominal_features].dtypes

In [ ]:
"""convert nominal features to type category, census_income_test"""
census_income_test[nominal_features] = census_income_test[nominal_features].astype('category')
census_income_test[nominal_features].dtypes

## *Exploratory Data Analysis*

### Objectives

- Print descriptive statistics for training and test sets
- Identify extreme values (outliers) in continuous features
- Visualize distribution of continuous features
- Print frequency distribution for nominal features
- Visualize frequency distribution for select nominal features
- Visualize frequency distribution of target
    - Consider distribution of target in training and test sets

In [ ]:
"""print info for clean train set"""

census_income_train.info()

In [ ]:
"""print info for clean test set"""
census_income_test.info()

In [ ]:
"""print summary statistics for continuous features in the training set"""
census_income_train[continuous_features].agg(func=['min', 'median', 'max', 'var', 'mean', 'std',
                                                   'skew', 'kurt'])

In [ ]:
"""print summary statistics for continuous features in the test set"""
census_income_test[continuous_features].agg(func=['min', 'median', 'max', 'mean', 'var', 'std',
                                                   'skew', 'kurt'])

Printing summary statistics for continuous features in the data shows the possible existnece of extreme values for features related to financials. A boxplot can be used to further confirm this assumption.

In [ ]:
"""plot high-level view of distribution continuous predictors train"""

fig, ax = plt.subplots(2, 2, figsize=(20, 15))
sns.boxplot(data=census_income_train, y='wage_per_hour', ax=ax[0,0])
sns.boxplot(data=census_income_train, y='capital_gains', ax=ax[0, 1])
sns.boxplot(data=census_income_train, y='capital_losses', ax=ax[1, 0])
sns.boxplot(data=census_income_train, y='dividends_from_stocks', ax=ax[1, 1])
#plt.ticklabel_format(style='Plain')
fig.suptitle('Distribution of Financial Features Train (High-Level)')
plt.tight_layout()

In [ ]:
"""plot high-level view of distribution continuous predictors test"""

fig, ax = plt.subplots(2, 2, figsize=(20, 15))
sns.boxplot(data=census_income_test, y='wage_per_hour', ax=ax[0,0])
sns.boxplot(data=census_income_test, y='capital_gains', ax=ax[0, 1])
sns.boxplot(data=census_income_test, y='capital_losses', ax=ax[1, 0])
sns.boxplot(data=census_income_train, y='dividends_from_stocks', ax=ax[1, 1])
#plt.ticklabel_format(style='Plain')
fig.suptitle('Distribution of Financial Features (High-Level)')
plt.tight_layout()

After visualizing outliers in features related to financials it's determined that records with financial features that have a z-score with absolute value greater than 3 should be identified. If these data repressent a significant amount of records in the training and test sets then records that have a z-score with absolute value greater than three should be retained. It's also worth noting that it's not uncommon for people in the US population to have income substantially greater than their peers.

In [ ]:
"""compute z-score for financial data training set and filter for z-scores with absolute value greater than 3"""
financial_features = ['wage_per_hour', 'capital_gains', 'capital_losses', 'dividends_from_stocks']
z_scores = stats.zscore(census_income_train[financial_features])
z_filter = (np.abs(z_scores > 3).any(axis=1))
census_income_train[z_filter].shape[0]/census_income_train.shape[0]*100

In [ ]:
"""compute z-score for financial data test set and filter for z-scores with absolute value greater than 3"""
financial_features = ['capital_gains', 'capital_losses', 'dividends_from_stocks']
z_scores = stats.zscore(census_income_test[financial_features])
z_filter = (np.abs(z_scores > 3).any(axis=1))
census_income_test[z_filter].shape[0]/census_income_test.shape[0]*100

In [ ]:
"""plot the distribution of people based attributes training set"""

fig, ax = plt.subplots(1, 3, figsize=(20, 15))
sns.histplot(data=census_income_train, x='age', stat='density', kde=True, ax=ax[0])
sns.histplot(data=census_income_train, x='num_persons_worked_for_employer', stat='density', kde=True, ax=ax[1])
sns.histplot(data=census_income_train, x='weeks_worked_in_year', stat='density', kde=True, ax=ax[2])

#plt.ticklabel_format(style='Plain')
fig.suptitle('Distribution of People Based Features Train (High-Level)')
plt.tight_layout()

In [ ]:
"""plot the distribution of people based attributes testing set"""

fig, ax = plt.subplots(1, 3, figsize=(20, 15))
sns.histplot(data=census_income_test, x='age', stat='density', kde=True, ax=ax[0])
sns.histplot(data=census_income_test, x='num_persons_worked_for_employer', stat='density', kde=True, ax=ax[1])
sns.histplot(data=census_income_test, x='weeks_worked_in_year', stat='density', kde=True, ax=ax[2])

#plt.ticklabel_format(style='Plain')
fig.suptitle('Distribution of People Based Features Test (High-Level)')
plt.tight_layout()

Plotting the distribution of continuous people based attributes in the data shows seems to show some extreme values in the data. That being said, the relatively normal values for skewness and kurtosis calcualted previously tell a different story. The data may be scaled for preprocessing depending upon the learning algorithm applied, but the effect of outliers should be relatively mild.

In [ ]:
"""print frequency distribution of nominal features training set"""

for feature in nominal_features:
    print(census_income_train.value_counts(feature, normalize=True)*100)
    print()


In [ ]:
"""plot frequency distribution of income level by education race and sex"""
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=census_income_train, x='outcome', hue='sex')
plt.title('Income Level by Race: Training Set')
plt.tight_layout()

In [ ]:
"""plot frequency distribution of income level by race"""
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=census_income_train, x='race', hue='sex')
plt.title('Gender by Race: Training Set')
plt.tight_layout()

In [ ]:
"""visualize the distribution of the target label in the training dataset"""

fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=census_income_train, x='outcome')
plt.title('Distribution of the Target Label: Training Set')
plt.tight_layout()

In [ ]:
"""visualize the distribution of the target label in the training dataset"""

fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=census_income_test, x='outcome')
plt.title('Distribution of the Target Label: Test Set')
plt.tight_layout()

## *_Data Preprocessing_*

*Objectives*

- Apply robust scaler to numeric features
- Encode outcome as binary classifier (0/1)

In [ ]:
"""apply preprocessing to the training and test set"""

#instantiate transformers
scaler = RobustScaler()
le = LabelEncoder()
#separate features and target
X_train = census_income_train.iloc[:, :-1]
y_train = census_income_train.iloc[:, -1:]
# apply transformers to training data
X_train[continuous_features] = scaler.fit_transform(X_train[continuous_features])
y_train['outcome'] = le.fit_transform(y_train)
#separate features and target
X_test = census_income_test.iloc[:, :-1]
y_test = census_income_test.iloc[:, -1:]
# pickle training data for future use
X_test.to_pickle('X_test')
y_test.to_pickle('y_test')
# apply transformers to test data
X_test[continuous_features] = scaler.fit_transform(X_test[continuous_features])
y_test['outcome'] = le.fit_transform(y_test)
# pickle test data for future use
X_test.to_pickle('X_test')
y_test.to_pickle('y_test')


## *Key Takeaways*

During data exploration it was determined there are extreme outliers present in features related to financial metrics for records. Considering how the data relate to the real-world problem being solved (prediction of income above or below 50K) outliers were not removed from the data. Instead a robust scaler was applied to numeric features to minimize the effects of outliers. Visualaizing the target (renamed "outcome" for the purpose of exploration) showed an imbalance of class membership in the data. This imbalance should be considered when applying supervised classification techniques to the data.